In [7]:
# nD k-Class Gausian Discriminant Analysis

import urllib2

import matplotlib.pyplot as plot
import numpy as np
import math
from sklearn import datasets, linear_model
from numpy.linalg import inv
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

data2 = urllib2.urlopen("http://mlr.cs.umass.edu/ml/machine-learning-databases/iris/iris.data")

# For Iris data
inp_data2 = []
files = np.loadtxt(data2,dtype=str, delimiter=',')

inp_data2 = np.array(files[:,0:-1], dtype='float')
givenY = files[:,-1]
target2=np.zeros(len(givenY), dtype='int')
unique_y = np.unique(givenY)

for cls in range(len(givenY)):
    for x in range(len(unique_y)):
        if(givenY[cls] == unique_y[x]):
            target2[cls] = x

data_matrix = np.matrix(np.array(inp_data2))
target_matrix = np.array(target2)

def covariance(train_data_clas, train_target_clas, className):

    class_matrix = []
    class_length = len(train_data_clas)
    classX = train_data_clas[train_target_clas==className]
        
    #find mean 
    matrix_mean = (np.array(classX)) 
    mean = [ sum(x)/len(matrix_mean) for x in zip(*matrix_mean) ]
    
    matrix = np.array(classX) 

    #find covariance
    sigma_matrix = np.zeros(shape=(matrix.shape[1],matrix.shape[1]))                                   
    for i in range(0, matrix.shape[1]):
        for j in range(0, matrix.shape[1]):
            sigma_matrix[i,j] =  np.mean((matrix[:,i:i+1]-mean[i]) * (matrix[:,j:j+1]-mean[j]))
    return sigma_matrix


def memberFunction(train_data_clas, train_target_clas, className, sigma):
    
    classX = train_data_clas[train_target_clas==className]
    
    # find mean
    matrix_mean = (np.array(classX)) 
    mean = [ sum(x)/len(matrix_mean) for x in zip(*matrix_mean) ]
    
    matrix = np.matrix(np.array(classX)) 
    
    class_length = len(matrix)
    total_length = len(train_data_clas)
    
    class_prob = float(class_length)/float(total_length)
    
    g1_x = []
    
    matrix = np.zeros(shape=(train_data_clas.shape[0],0)) 
    for i in range(total_length):
        value = -math.log(abs(np.linalg.det(sigma))) - ((np.matrix(train_data_clas[i][0] - mean)) * (sigma**-1)* (np.matrix(train_data_clas[i][0] - mean).T))/2 
        + math.log(class_prob)
        
        g1_x.append(value)
    
    return g1_x


def discriminant_function(max_gX, clas):
    
    diff = []
    predicted = []
    array = []

    for index in range(len(clas)):
        array.append(max_gX[index])
    predicted_values = np.maximum.reduce(array)
        
    for x in range(len(predicted_values)):
        for y in max_gX:
            key = max_gX[y]
            if(key[x] == predicted_values[x]):
                predicted.append(y)
    return predicted

def findOtherParameters(confusion_mat):
    
    list_diagonal = np.zeros(confusion_mat.shape[0])
    list_row_sum = np.zeros(confusion_mat.shape[0])
    list_column_sum=np.zeros(confusion_mat.shape[1])
    
    precision_value = []
    recall_value = []
    f_measure_value = []
    
    total = np.sum(confusion_mat)
    confuse_diagonal = 0

    for i in range(confusion_mat.shape[0]):
        for j in range(confusion_mat.shape[1]):
            list_row_sum[i] += confusion_mat[i][j]
            list_column_sum[i] += confusion_mat[j][i]
            if(i==j):
                list_diagonal[i] = confusion_mat[i][j]
                confuse_diagonal +=  confusion_mat[i][j]
    
    accuracy = float(confuse_diagonal)/total

    for index in range(len(list_row_sum)):
        if list_row_sum[index]==0:
            precision_value.append(0.0)
        else:
            precision_value.append((float)(list_diagonal[index]) / list_row_sum[index])
            
        if list_column_sum[index]==0:
            recall_value.append(0)
        else:
            recall_value.append((float)(list_diagonal[index]) / list_column_sum[index])
        
        if precision_value[index]==0 or recall_value[index]==0:
            f_measure_value.append(0)
        else:
            f_measure_value.append((float) (2 * precision_value[index] * recall_value[index]) / (precision_value[index] + recall_value[index]))
        
    return accuracy, precision_value, recall_value, f_measure_value

print "\nK-Fold started "        
max_gX = {}

maximum_gX = []
kf = KFold(data_matrix.shape[0], n_folds=10, shuffle=True)
print "No of folds = ",len(kf)

temp = 0
final_precision=0
final_recall = 0
final_fmeasure = 0
final_accuracy = 0

for train_index, test_index in kf:
    X_Train_Data, X_Test_Data = data_matrix[train_index], data_matrix[test_index]
    Y_Train_Data, Y_Test_Data = target_matrix[train_index], target_matrix[test_index]
    
    # iterate data for each class
    for clas in np.unique(target_matrix):
        sigma_clas = covariance(X_Train_Data, Y_Train_Data, clas)
        memberFun_clas = memberFunction(X_Test_Data, Y_Test_Data, clas, sigma_clas)
        max_gX.update({int(clas):memberFun_clas})

    # find discriminanent function
    disc_function = discriminant_function(max_gX, np.unique(target_matrix))
    
    # confusion matrix
    #print "confusion matrix"
    confusion_mat = confusion_matrix(Y_Test_Data, disc_function)
    #print confusion_mat
    
    # find precision, recall , f-measure
    accuracy, precision_val, recall_val, f_measure_val = findOtherParameters(confusion_mat)
     
    # print errors
    """
    print "Precision = ", precision_val
    print "Recall = ", recall_val
    print "F-Measure", f_measure_val
    print "\n Fold Completed \n"

    """
    if accuracy>temp:
        if(accuracy !=1):
            final_accuracy = accuracy
            final_precision = precision_val
            final_recall = recall_val
            final_fmeasure = f_measure_val
            temp = accuracy

    print "\n K-fold completed \n"
    
# print errors
print "Final Output : "
print "Accuracy = ", final_accuracy
print "Precision = ", final_precision
print "Recall = ", final_recall
print "F-Measure", final_fmeasure
print "\n Fold Completed \n"


K-Fold started 
No of folds =  10

 K-fold completed 


 K-fold completed 


 K-fold completed 


 K-fold completed 


 K-fold completed 


 K-fold completed 


 K-fold completed 


 K-fold completed 


 K-fold completed 


 K-fold completed 

Final Output : 
Accuracy =  0.933333333333
Precision =  [1.0, 1.0, 0.80000000000000004]
Recall =  [1.0, 0.80000000000000004, 1.0]
F-Measure [1.0, 0.88888888888888895, 0.88888888888888895]

 Fold Completed 

